<a href="https://colab.research.google.com/github/robrocha42/E-Commerce_Shipping_Data/blob/main/E_Commerce_Shipping_Data_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# E-Commerce Shipping Data - Kaggle - Orientado a objetos

In [13]:
#Classe DataPrep
#E-Commerce Shipping Data - Site Kaggle

# tratamento de daos
import pandas as pd
# machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

class DataPrep:
  def __init__(self, data):
    "Instancia a classe com o conjunto de dados que precisa ser processado" 
    self.data = data

  #para ser utilizado sem instanciar o objeto
  @staticmethod
  def _one_hot_encoder(df: pd.DataFrame, variavel: str) -> pd.DataFrame:
    "Transforma uma variável categórica utilizando One Hot Encoding" 
    dummies = pd.get_dummies (df [variavel], prefix=variavel)
    df = pd.concat([df, dummies], axis=1)
    df.drop(columns=variavel, inplace=True)
    return df

  def _tratar_variaveis_categoricas (self) -> None:
    "Usar One Hot Encoder para tratar as variáveis categóricas" 
    self.data = DataPrep._one_hot_encoder( self.data, 'Warehouse_block')
    self.data = DataPrep._one_hot_encoder( self.data, 'Mode_of_Shipment')
    self.data = DataPrep._one_hot_encoder( self.data, 'Product_importance')
    self.data = DataPrep._one_hot_encoder( self.data, 'Gender')

  def _remover_variaveis (self) -> None:
    "Remove as variáveis que não serão utilizadas pelo modelo." 
    self.data.drop(columns='ID', inplace=True)

  def _normalizar_dados(self) -> None:
    variaveis = self.data.drop(columns='Reached.on.Time_Y.N')
    var_cols = variaveis.columns
    resposta = self.data['Reached.on.Time_Y.N']

    scaler = MinMaxScaler()
    variaveis = scaler.fit_transform(variaveis)
    variaveis = pd.DataFrame(variaveis, columns=var_cols)

    self.data = pd.concat([variaveis, resposta], axis=1)

  def _separar_treino_teste (self):
    "Divide o dataset em conjunto de treino e teste"
    treino, teste = train_test_split(
      self.data,
      test_size=0.3,
      random_state=2023)
    return treino, teste

  def preparar_dados (self):
    "Aplica todas as transformações de dados."
    self._tratar_variaveis_categoricas()
    self._remover_variaveis()
    self._normalizar_dados()
    treino, teste = self._separar_treino_teste() 
    return treino, teste

In [14]:
#Treinar um classificador para prever se uma determinada entrega chegará dentro do prazo
#Clase para modelagem de dados

#machine learning
from sklearn.linear_model import LogisticRegression

class Model:
  def __init__(self, df_treino: pd.DataFrame, df_teste: pd.DataFrame) -> None:
    self.df_treino = df_treino
    self.df_teste = df_teste
    #clf - classificador 
    self.clf = LogisticRegression()

  def separar_variaveis(self):
    self.X_treino = self.df_treino.drop(columns='Reached.on.Time_Y.N')
    self.Y_treino = self.df_treino['Reached.on.Time_Y.N']

    self.X_teste = self.df_teste.drop(columns='Reached.on.Time_Y.N')
    self.Y_teste = self.df_teste['Reached.on.Time_Y.N']

  #Métodos para treinar e avaliar o modelo e fazer previsões
  def fit(self):
    self.separar_variaveis()
    self.clf.fit(self.X_treino, self.Y_treino)

  def predict(self, X: pd.DataFrame):
    predictions = self.clf.predict(X)
    return predictions

  def score(self):
    return self.clf.score(self.X_teste, self.Y_teste)


In [15]:
df = pd.read_csv('/content/Train.csv') #Caminho do arquivo
dp = DataPrep(df)

# Toda a preparação de dados necessaria e retorna os DataFrames de treino e teste já separados
df_treino, df_teste = dp.preparar_dados()

modelo = Model(df_treino, df_teste)
modelo.fit()
#64,6% de acurácia
modelo.score()

0.6460606060606061